# Chapter 11 - Structural Patterns

- Decorator and composite design patterns help to complext oject structures
- Adapter pattern helps to connect an existing interface to another incompatible interface - enabling 2 systems to communicate
- The Faade pattern provides a simpler interface to a complex system



## The Decorator design pattern

- Dynamically add new functionality to an object by wrapping it with 1 or more decorator objects (At runtime)
- Helps to follow the open-closed principle


In [20]:
classDiagram 
    direction
    class IComponent {
        <<interface>>
        +Operation()
    }
    class ComponentA {
        +Operation()
    }
    class DecoratorA {
        -component IComponent
        +DecoratorA(component IComponent)
        +Operation()
        -AddBehaviorA()
    }

    note for IComponent "Client"

    ComponentA ..|> IComponent
    DecoratorA ..|> IComponent
    DecoratorA o-- IComponent

In [27]:
sequenceDiagram
    actor Client
    participant DecoratorA
    participant ComponentA
    activate Client
    Client->>DecoratorA: Operation()
    activate DecoratorA
    DecoratorA->>ComponentA: Operation()
    activate ComponentA
    ComponentA-->>DecoratorA: Return
    deactivate ComponentA
    DecoratorA->>DecoratorA: AddBehaviorA()
    DecoratorA-->>Client: Return
    deactivate DecoratorA
    deactivate Client

- Decorators can be nested

In [1]:
public interface IComponent
{
 string Operation();
}

public class ComponentA : IComponent
{
    public string Operation()
    {
        return "Hello from ComponentA";
    }
}

public class DecoratorA : IComponent
{
    private readonly IComponent _component;

    public DecoratorA(IComponent component)
    {
        _component = component ?? throw new ArgumentNullException(nameof(component));
    }
 
    public string Operation()
    {
        var result = _component.Operation();
        return $"<DecoratorA>{result}</DecoratorA>";
    }
}

public class DecoratorB : IComponent
{
    private readonly IComponent _component;

    public DecoratorB(IComponent component)
    {
        _component = component ?? throw new ArgumentNullException(nameof(component));
    }

    public string Operation()
    {
        var result = _component.Operation();
        return $"<DecoratorB>{result}</DecoratorB>";
    }
}

In [4]:
//builder.Services.AddSingleton<IComponent, ComponentA>();
//app.MapGet("/", (IComponent component) => component.Operation());
var component = new ComponentA();

component.Operation().Display();

Hello from ComponentA

In [7]:
//builder.Services.AddSingleton<IComponent>(serviceProvider => new DecoratorA(newComponentA()));
//app.MapGet("/", (IComponent component) => component.Operation());
var component = new DecoratorA(new ComponentA());

component.Operation().Display();

<DecoratorA>Hello from ComponentA</DecoratorA>

In [8]:
//builder.Services.AddSingleton<IComponent>(serviceProvider => new DecoratorB(new DecoratorA(new ComponentA())));
//app.MapGet("/", (IComponent component) => component.Operation());
var component = new DecoratorB(new DecoratorA(new ComponentA()));

component.Operation().Display();

<DecoratorB><DecoratorA>Hello from ComponentA</DecoratorA></DecoratorB>

### Use Scrutor

[Scrutor](https://github.com/khellang/Scrutor) can be used to simplify the composition of decorators.

```bash
dotnet add package Scrutor
```

```csharp
builder.Services
    .AddSingleton<IComponent, ComponentA>()
    .Decorate<IComponent, DecoratorA>()
    .Decorate<IComponent, DecoratorB>();
```

## The Composite design pattern

- Helps manage complext object structures
- i.e graph or tree with self managing nodes



### Diagram

- *Composite* collection of `IComponent`
    * calls `Operation()` on each of its children
- *Component* represents a single element; a leaf


In [1]:
classDiagram
    direction LR
    class IComponent {
        <<interface>>
        +Operation()
    }
    class Component {
        +Operation()
    }
    class Composite {
        +children IComponent[]
        +Operation()
    }


    IComponent <|-- Component
    IComponent <|-- Composite
    Composite o-- IComponent : children

### C# Example: Composite Pattern for Organizational Hierarchy

The following example demonstrates how to use the Composite design pattern to represent an organizational hierarchy with employees and managers.

In [17]:
public interface IEmployee
{
    void Display(int indent = 0);
}

public class Employee : IEmployee
{
    public string Name { get; }
    public string Position { get; }

    public Employee(string name, string position)
    {
        Name = name;
        Position = position;
    }

    public void Display(int indent = 0)
    {
        Console.WriteLine(new string(' ', indent) + $"- {Position}: {Name}");
    }
}

public class Manager : IEmployee
{
    public string Name { get; }
    public string Position { get; }
    private readonly List<IEmployee> _subordinates = new();

    public Manager(string name, string position)
    {
        Name = name;
        Position = position;
    }

    public void Add(IEmployee employee)
    {
        _subordinates.Add(employee);
    }

    public void Remove(IEmployee employee)
    {
        _subordinates.Remove(employee);
    }

    public void Display(int indent = 0)
    {
        Console.WriteLine(new string(' ', indent) + $"+ {Position}: {Name}");
        foreach (var subordinate in _subordinates)
        {
            subordinate.Display(indent + 2);
        }
    }
}

var ceo = new Manager("Alice", "CEO");
var headDev = new Manager("Bob", "Head of Development");
var dev1 = new Manager("Charlie", "Team Lead");
var dev2 = new Employee("Diana", "Developer");
var headSales = new Manager("Eve", "Head of Sales");
var sales1 = new Employee("Frank", "Salesperson");

headDev.Add(dev1);
dev1.Add(dev2);
headSales.Add(sales1);
ceo.Add(headDev);
ceo.Add(headSales);

ceo.Display();


+ CEO: Alice
  + Head of Development: Bob
    + Team Lead: Charlie
      - Developer: Diana
  + Head of Sales: Eve
    - Salesperson: Frank


This code defines an `IEmployee` interface, concrete `Employee` (leaf), and `Manager` (composite) classes. Managers can have subordinates (other employees or managers), and the hierarchy can be displayed recursively.

See Bookstore example in [C11/src/Composite](../C11/src/Composite/Program.cs)

## The Adapter design pattern

- Used to make two incompatible interfaces work together without modifying their existing code

### Diagram

- Adapter calls `SomeMethod()` on Adaptee
- You can also use inheritance if access to private members is needed but prefer composition if possible

In [6]:
classDiagram
    direction LR
    class ITarget {
        <<interface>>
        +Operation()
    }
    class Client {

    }
    class Adapter {
        +adaptee Adaptee
        +Operation()
    }
    class Adaptee {
        +SomeMethod()
    }


    Client o-- ITarget
    Adapter o-- Adaptee
    Adapter ..|> ITarget


### Project: Greeter

In [7]:
//External class we want to use (Adaptee)
public class ExternalGreeter
{
    public string GreetByName(string name)
    {
        return $"Adaptee says: hi {name}!";
    }
}

//Interface used in our system
public interface IGreeter
{
    string Greeting();
}

//Adapter class that adapts the ExternalGreeter to the IGreeter interface
public class ExternalGreeterAdapter : IGreeter
{
    private readonly ExternalGreeter _adaptee;

    public ExternalGreeterAdapter(ExternalGreeter adaptee)
    {
        _adaptee = adaptee ?? throw new ArgumentNullException(nameof(adaptee));
    }

    public string Greeting()
    {
        return _adaptee.GreetByName("ExternalGreeterAdapter");
    }
}

// Usage
// builder.Services.AddSingleton<ExternalGreeter>();
// builder.Services.AddSingleton<IGreeter, ExternalGreeterAdapter>();
var adaptee = new ExternalGreeter();
var adapter = new ExternalGreeterAdapter(adaptee);
adapter.Greeting().Display();

Adaptee says: hi ExternalGreeterAdapter!

## The Facade design pattern

- Simplifies access to a complex system (Could be multiple sub-systems)

In [14]:
classDiagram
    direction LR
    class Client {
        +UseFacade()
    }
    class Facade {
        +OperationA()
        +OperationB()
    }
    %% SubsystemA namespace
    class SubsystemAClass1 {
        +ActionA1()
    }
    class SubsystemAClass2 {
        +ActionA2()
    }
    SubsystemAClass1 <..> SubsystemAClass2 : collaborates
    %% SubsystemB namespace
    class SubsystemBClass1 {
        +ActionB1()
    }

    Client o-- Facade : uses
    Facade o-- SubsystemAClass1 : uses
    Facade o-- SubsystemAClass2 : uses
    Facade o-- SubsystemBClass1 : uses

- **Opaque facades**: Facade is inside the subsystem, all other classes have an internal visibility modifier
- **Transparent facades**: Subsytem has a public modifier so the Facade and be internal or external
- Static - avoid if possible

### Project

Subsystem

In [15]:
//Namespace: MySubsystem

//Abstractions
public interface IInventoryService
{
    bool CheckStock(string productId, int quantity);
}

public interface IShippingService
{
    void ScheduleShipping(int orderId);
}

public interface IOrderProcessingService
{
    int CreateOrder(string productId, int quantity);
    string GetOrderStatus(int orderId);
}

//Implementations
public class InventoryService : IInventoryService
{
    public bool CheckStock(string productId, int quantity)
    {
        // Check if the product is available in the desired quantity
        return true; // Simplified for example
    }
}

public class OrderProcessingService : IOrderProcessingService
{
    public int CreateOrder(string productId, int quantity)
    {
        // Logic to create an order
        return 123; // Returns a mock order ID
    }

    public string GetOrderStatus(int orderId)
    {
        // Logic to get order status
        return "Order Shipped"; // Simplified for example
    }
}

public class ShippingService : IShippingService
{
    public void ScheduleShipping(int orderId)
    {
        // Logic to schedule shipping
    }
}




Facade

In [16]:
//using MySubsystem;

public interface IECommerceFacade
{
    string PlaceOrder(string productId, int quantity);
    string CheckOrderStatus(int orderId);
}

public class ECommerceFacade : IECommerceFacade
{
    private readonly IInventoryService _inventoryService;
    private readonly IOrderProcessingService _orderProcessingService;
    private readonly IShippingService _shippingService;

    public ECommerceFacade(IInventoryService inventoryService, IOrderProcessingService orderProcessingService, IShippingService shippingService)
    {
        _inventoryService = inventoryService ?? throw new ArgumentNullException(nameof(inventoryService));
        _orderProcessingService = orderProcessingService ?? throw new ArgumentNullException(nameof(orderProcessingService));
        _shippingService = shippingService ?? throw new ArgumentNullException(nameof(shippingService));
    }

    public string PlaceOrder(string productId, int quantity)
    {
        if (_inventoryService.CheckStock(productId, quantity))
        {
            var orderId = _orderProcessingService.CreateOrder(productId, quantity);
            _shippingService.ScheduleShipping(orderId);
            return $"Order {orderId} placed successfully.";
        }
        return "Order failed due to insufficient stock.";
    }

    public string CheckOrderStatus(int orderId)
    {
        return _orderProcessingService.GetOrderStatus(orderId);
    }
}



In [17]:
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.Extensions.Hosting.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Mvc.ViewFeatures.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Diagnostics.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.Results.dll"

In [20]:
using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;


public static IServiceCollection AddFacadeSubSystem(this IServiceCollection services)
{
    services.TryAddSingleton<IInventoryService, InventoryService>();
    services.TryAddSingleton<IOrderProcessingService, OrderProcessingService>();
    services.TryAddSingleton<IShippingService, ShippingService>();
    services.TryAddSingleton<IECommerceFacade, ECommerceFacade>();
    return services;
}

In [24]:
using Microsoft.AspNetCore.Http;
using Microsoft.AspNetCore.Http.HttpResults;
using Microsoft.AspNetCore.Routing;
using Microsoft.AspNetCore.Builder;
using Microsoft.Extensions.DependencyInjection;

// Define the command line arguments
string[] args = {"--urls","http://localhost:7001"};

var builder = WebApplication.CreateBuilder(args);
builder.Services.AddFacadeSubSystem();
var app = builder.Build();

app.MapPost( "/PlaceOrder", (PlaceOrder order, IECommerceFacade eCommerceFacade) => 
    eCommerceFacade.PlaceOrder(order.ProductId, order.Quantity)
);

app.MapGet(
 "/CheckOrderStatus/{orderId}",
 (int orderId, IECommerceFacade eCommerceFacade)
 => eCommerceFacade.CheckOrderStatus(orderId)
);

app.RunAsync();

public record class PlaceOrder(string ProductId, int Quantity);

info: Microsoft.Hosting.Lifetime[14]
      Now listening on: http://localhost:7001
info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Production
info: Microsoft.Hosting.Lifetime[0]
      Content root path: c:\Users\jason\training\dotnet\Architecting-ASP.NET-Core-Applications-3E\MyNotes


In [22]:
using System.Net.Http;

var httpClient = new HttpClient();

var response = await httpClient.PostAsync("http://localhost:7001/PlaceOrder",
    new StringContent(
        """{"ProductId":"12345","Quantity":2}""",
        System.Text.Encoding.UTF8,
        "application/json"
    )
);  
response.EnsureSuccessStatusCode();
var content = await response.Content.ReadAsStringAsync();
content.DisplayAs("application/json")

In [25]:
using System.Net.Http;

var httpClient = new HttpClient();

var response = await httpClient.GetAsync("http://localhost:7001/CheckOrderStatus/123");
response.EnsureSuccessStatusCode();
var content = await response.Content.ReadAsStringAsync();
content.DisplayAs("application/json")

In [26]:
app.StopAsync();